In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
input_size = 784 # image pixels
hidden_size = 400 # input size // 2 better
out_size = 10 # 0-9 outputs mnist
epochs = 10
batch_size = 100
learning_rate = 0.001

In [3]:
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./data',
                               train=False,
                               transform=transforms.ToTensor())


100%|██████████| 9912422/9912422 [00:00<00:00, 71683002.31it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 43062813.30it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 38109334.05it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 3037877.34it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [5]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, out_size):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, out_size)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.relu(out)
    out = self.fc3(out)
    return out

In [6]:
net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()

if CUDA:
  net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)


In [8]:
net.parameters

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=10, bias=True)
)>

In [ ]:
# Training
correct_train = 0
total_train = 0
for epoch in range(epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = Variable(images.view(-1, 28*28))
    labels = Variable(labels)
    if CUDA:
      images = images.cuda()
      labels = labels.cuda()

    optimizer.zero_grad()
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1) # Return the second argument of torch.max which represents the index location of each max value found

    total_train += labels.size(0)

    if CUDA:
      correct_train += (predicted.cpu() == labels.cpu()).sum() # not available on gpu
    else:
      correct_train += (predicted == labels).sum()

    loss = criterion(outputs, labels)
    loss.backward() # back prop
    optimizer.step() # update weights

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}], Iteration [{}/{}], Training Loss: {}, Training Accuracy: {}%'.format(
          epoch + 1, epochs, i + 1, len(train_dataset)// batch_size, loss.data, (100 * correct_train / total_train)
      ))

print('Done Training')


In [19]:
acc = (correct_train / total_train).item()
acc

0.986163318157196

In [20]:
# Testing
correct, total = 0, 0
for images, labels in test_loader:
  images = Variable(images.view(-1, 28 * 28))
  if CUDA:
    images = images.view()
  outputs = net(images)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  if CUDA:
    correct += (predicted.cpu() == labels.cpu()).sum()
  else:
    correct += (predicted == labels).sum()

print('Test Accuracy: %d %%' %(100 * correct/ total) )

Test Accuracy: 97 %
